# References

Simple Code: 

1. http://peterbloem.nl/blog/transformers
2. https://github.com/feather-ai/transformers-tutorial
3. https://jalammar.github.io/illustrated-transformer/

Optimized Multi-GPU (For version 2):
http://nlp.seas.harvard.edu/2018/04/03/attention.html



# Self-Attention and Multi-Headed Attention

We have input in the form of word embeddings $ x_{i}, x_{j},....x_{n} $

We want the output $y_{i}$ such that it tells us on which $x$ it focused.

For ex: The animal didn't cross the street because it was too tired.

In Language Modelling, when the NN is generating the word "it" what did the NN referred to, the "animal" or the "street".

So we want to know the "attention" the NN paid to each of the word in the sentence while generating the word "it".

In its simplest form this attention can be formaulated as dot product since it captures the similarity between two vectors.

And this similarity could work as the weight for each of the word $j$ when the NN generated the word $i$

So at any point of time $i$ the output $y_{i}$ can be given by $$ y_{i} = \sum_{j} w_{ij} x_{i} $$

where $$ w_{ij} = x_{i}^T x{j} $$ a.k.a the dot product with every other vector

Now we can simply apply softmax to keep the values as probabilities.


Let's see how we can implement this in pytorch.

In [1]:
def self_attention(x):
    """Function to apply self attention to the vector x.
    Arguments
    -------------------------
    Shape of x: batch_size, t, k where t is the no. of vectors 
                and k is the size of each vector
                i.e. x contains all the word embedding vectors

    Returns:
    --------------------------


    Notes:
    --------------------------
    Assume we have a tensor of size (batch_size, t vectors, k (dimension of each vector))
    k is fixed as the size of the embedding layer and the encoding layer is fixed
    so for each vector we have the same size.
    """
    # torch.bmm is batched matrix multiplication
    # this is basically xx' for calculating weights wij.
    raw_weights = torch.bmm(x, x.transpose(1,2))

    # Turning these weights into probabilites by applying row-wise softmax
    weights = F.softmax(raw_weights, dim=2)

    # y = wx to get the output vector of size (b, t, k)
    y = torch.bmm(weights,x)

    return y

## query, keys and values?

In the self-attention stage, every input vector is used in three ways.</br>
    1. Compared to every other vector to establish the weights of its own output. (Query) </br>
    2. Compared to every other vector to establish the weights for output of jth vector Yj. (Key)</br>
    3. It is used as part of the weighted sum to compute each output vector 
       once the weights are established. (Value)</br>
       **Need more clearance on the use of Values**
    
To get to these vectors or to make them play these roles, we introduce new parameters in the form of three matrices.

In other words we add three $k x k$ (k is embedding dimension) matrices called $W_{q}$, $W_{k}$, $W_{v}$ and calcualte three *linear transformations* of the input vector $x{i}$

$$ q_{i} = W_q x_i  ~~~~~ k_i = W_k x_i ~~~~~~~~~ v_i = W_v x_i $$

Again, $q_i$ is weights of its **own** output, $k_i$ is with other

$$ w_{ij}' = q_{i}' k_{j} $$

$$ w_{ij} = softmax(w_{ij}') $$

$$ y_i = \sum_{j} W_{ij}v_j$$


## Scaling the dot product

Softmax is sensitive to very large values and the  average value of the dot product increases as the embedding dimension $k$ increases which kills the gradients and stops training. 

Solution: Scale the dot product

How? 

\begin{equation}
  w_{ij}' = \frac{q_{i}^T k_{j}}{\sqrt{k}}
\end{equation}

why $\sqrt{k}$? Imagine a vector in $ℝ^k$ with values all c. Its Euclidean length is k$\sqrt{c}$. Therefore, we are dividing out the amount by which the increase in dimension increases the length of the average vectors.



## Let's move to multi-head attention

A word can mean different things to different neighbors.

Ex: Mary gave roses to susan. The word "gave" contains different meanings for Mary and roses and Susan. Mary is the giver and susan is the receiver. With simple attention it will sum up all the weights together once and that's it.

In multiple attention, we apply this attention multiple times for the same word so that the same word "gave" can apply different weight to Mary in different scenarios. These are called attention heads and are represented by "r".

So the new matrices will be $W_{q}^r$, $W_{k}^r$, and $W_{v}^r$

Now we pass an input $x_i$ through all these attention heads and finally **concatenate** them and pass them through a linear layer to reduce the dimension back to the embedding dimension $k$.

Although if we simply make multiple copies of a single attention with their own query, key and value then it will be much too computationally expensive.

A better way to do this is to cut the input vector into chunks of size equal to $k$/no_of_attention_heads.


Let's code now!




In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

class SelfAttention(nn.module):
    '''
    Implementation of a simple multi-head attention
    
    Note: This is not the optimized version, here we will have n_heads copies of 
    query, key and value.
    
    '''
    
    def __init__(self, k:int, n_heads:int = 8):
        '''
        Args:
        ---------------------------
        k: Embedding dimension 
        n_heads: Number of multi-attention heads
        
        Notes:
        ---------------------------
        We think of the h attention heads as h separate sets of three matrices 
        𝐖_q^r, 𝐖_k^r,𝐖_v^r, but it's actually more efficient to combine these for 
        all heads into three single k×hk matrices, 
        so that we can compute all the concatenated queries, keys and values 
        in a single matrix multiplication.
        
        '''
        super().__init__()
        self.k = k 
        self.n_heads = n_heads
        # These compute the queries, keys and values for all 
        # heads (as a single concatenated vector)
        # this will generate the weight matrices Wq^r, Wk^r, Wv^r
        self.tokeys    = nn.Linear(k, k * n_heads, bias=False)
        self.toqueries = nn.Linear(k, k * n_heads, bias=False)
        self.tovalues  = nn.Linear(k, k * n_heads, bias=False)

        # This unifies the outputs of the different heads into 
        # a single k-vector
        self.unifyheads = nn.Linear(n_heads * k, k)
        
        def forward(self, x):
            '''
            Shape of x: batch_size, t, k where t is the no. of vectors 
            and k is the size of each vector
            i.e. x contains all the word embedding vectors
            '''
            b, t, k = x.size()
            h = self.n_heads
            # view is similar to numpy's reshape but it doesn't create any copies
            # The output of each of the linear layers will be 
            # b, t, h*k which we are reshaping as (b,t,h,k)
            queries = self.toqueries(x).view(b, t, h, k)
            keys    = self.tokeys(x).view(b, t, h, k)
            values  = self.tovalues(x) .view(b, t, h, k)
            
            # Next, we need to compute the dot products. This is the same operation
            # for every head, so we fold the heads into the batch dimension. 
            # This ensures that we can use torch.bmm() as before, 
            # and the whole collection of keys, queries and values 
            # will just be seen as a slightly larger batch.
            # Since the head and batch dimension are not next to each other,
            # we need to transpose before we reshape. 
            # (This is costly, but it seems to be unavoidable.)
            # contiguous means to make a copy of data so that the order of elements 
            # remains unchanged inspite of using view.
            queries = queries.transpose(1,2).contiguous().view(b*h,t,k)
            keys = queries.transpose(1,2).contiguous().view(b*h,t,k)
            values = queries.transpose(1,2).contiguous().view(b*h,t,k)
            
            # let's scale these before doing dot product.
            # instead scale the keys and queries by fourth root of k before multiplying them together. 
            # This should save memory for longer sequences.
            queries = queries / (k ** (1/4))
            keys    = keys / (k ** (1/4))
            
            
            # - get dot product of scaled queries and keys
            dot = torch.bmm(queries, keys.transpose(1, 2))
            # - dot has size (b*h, t, t) containing raw weights

            dot = F.softmax(dot, dim=2) 
            # - dot now contains row-wise normalized weights
            
            # apply the self attention to the values
            out = torch.bmm(dot, values).view(b, h, t, k)
            
            # swap h, t back, unify heads
            out = out.transpose(1, 2).contiguous().view(b, t, h * k)
            return self.unifyheads(out)

# Transformer

Any architecture designed to process a connected set of units—such 
as the tokens in a sequence or the pixels in an image—where the only 
interaction between units is through self-attention.

![Transfomer Block](http://peterbloem.nl/files/transformers/transformer-block.svg)

the block applies, in sequence: a self attention layer, layer normalization, a feed forward layer (a single MLP applied independently to each vector), and another layer normalization.

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, k, n_heads):
        super().__init__()

        self.attention = SelfAttention(k, n_heads=heads)

        self.norm1 = nn.LayerNorm(k)
        self.norm2 = nn.LayerNorm(k)
        
        # We’ve made the relatively arbitrary choice of making the hidden layer 
        # of the feedforward 4 times as big as the input and output. 
        # Smaller values may work as well, and save memory, 
        # but it should be bigger than the input/output layers.
        self.ff = nn.Sequential(
          nn.Linear(k, 4 * k),
          nn.ReLU(),
          nn.Linear(4 * k, k))

    def forward(self, x):
        attended = self.attention(x)
        x = self.norm1(attended + x)
        fedforward = self.ff(x)
        return self.norm2(fedforward + x)

## Let's build a classifier using Transformers

![Sentiment Classification](http://peterbloem.nl/files/transformers/classifier.svg)

**position embeddings** </br>
We simply embed the positions like we did the words. Just like we created embedding vectors 𝐯cat and 𝐯susan, we create embedding vectors 𝐯12 and 𝐯25. Up to however long we expect sequences to get. The drawback is that we have to see sequences of every length during training, otherwise the relevant position embeddings don't get trained. The benefit is that it works pretty well, and it's easy to implement.</br>

**position encodings**</br>
Position encodings work in the same way as embeddings, except that we don't learn the position vectors, we just choose some function f:ℕ→ℝk to map the positions to real valued vectors, and let the network figure out how to interpret these encodings. The benefit is that for a well chosen function, the network should be able to deal with sequences that are longer than those it's seen during training (it's unlikely to perform well on them, but at least we can check). The drawbacks are that the choice of encoding function is a complicated hyperparameter, and it complicates the implementation a little.


For the sake of simplicity, we’ll use position embeddings in our implementation.

In [ ]:
class Transformer(nn.Module):
    def __init__(self, k, heads, depth, seq_length, num_tokens, num_classes):
        super().__init__()

        self.num_tokens = num_tokens
        self.token_emb = nn.Embedding(num_tokens, k)
        self.pos_emb = nn.Embedding(seq_length, k)

        # The sequence of transformer blocks that does all the 
        # heavy lifting
        tblocks = []
        for i in range(depth):
            tblocks.append(TransformerBlock(k=k, heads=heads))
        
        self.tblocks = nn.Sequential(*tblocks)

        # Maps the final output sequence to class logits
        self.toprobs = nn.Linear(k, num_classes)
        
    def forward(self, x):
        """
        :param x: A (b, t) tensor of integer values representing 
                  words (in some predetermined vocabulary).
        :return: A (b, c) tensor of log-probabilities over the 
                 classes (where c is the nr. of classes).
        """
        # generate token embeddings
        tokens = self.token_emb(x)
        b, t, k = tokens.size()

        # generate position embeddings
        positions = torch.arange(t)
        positions = self.pos_emb(positions)[None, :, :].expand(b, t, k)
        
        x = tokens + positions
        x = self.tblocks(x)
        
        # Average-pool over the t dimension and project to class 
        # probabilities
        x = self.toprobs(x.mean(dim=1))
        return F.log_softmax(x, dim=1)

# What is Position Encoding?

Intuition: https://www.youtube.com/watch?v=1biZfFLPRSY
Code: https://www.youtube.com/watch?v=LSCsfeEELso

We add the position information to the embeddings because unlike RNN there is no position (or time) in Transformers.

The input to the NLP models is usually an embedding of the words. So the logical thing to do is to add this positional information in the embeddings itself instead of the actual input.

Note: We are not simply adding the position of the word in the sentence, rather treat this as an embedding which adds semantic information to the vectors.

The way the authors proposed this is by adding a vector to the embedding vector.

So to add these embedding and posistional encoding (PE) vectors, we need same dimensions.

Now consider the vector PE of dimension d = embedding dimension,

the positions are encoded using Sinousuidals.

$$ PE_{pos,2i} = sin(\frac{pos}{10000^{2i/d}}) $$ and

$$ PE_{pos,2i+1} = cos(\frac{pos}{10000^{2i/d}}) $$

where pos is the position of the word and i is the index of d-dimensional vector.

Why sinosuidal? 
a. Its always between 0 and 1 so your positional encoding never empowers the embedding which contains semantic information.
b. Its infinite so the sequence length never becomes an issue.
c. In comparison to sigmoid, sin and cosine have veriability for big numbers as well, while the sigmoid becomes 1 after a fixed number.